In [1]:
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import math
%matplotlib inline

2022-06-29 13:52:12.824955: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-06-29 13:52:12.825080: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2022-06-29 13:52:18.647531: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-29 13:52:18.650640: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-06-29 13:52:18.650679: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-29 13:52:18.650706: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d49ecccbca92): /proc/driver/nvidia/version does not exist
2022-06-29 13:52:18.655319: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

REPLICAS:  8


In [3]:
BATCH_SIZE=32 * strategy.num_replicas_in_sync
GCS_PATH=KaggleDatasets().get_gcs_path('shop-tpu-folds')
MAX_LENGTH=180
PRE_TRAINED_NAME="bert-base-uncased"

In [4]:
df=pd.read_csv(f"{GCS_PATH}/fold_data.csv")
df.head()

,posting_id,image,image_phash,title,label_group,image_path,gfold
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,train_images\0000a68812bc7e98c42888dfb1c07da0.jpg,0
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",7572,train_images\00039780dfc94d01db8676fe789ecd05.jpg,2
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,train_images\000a190fdd715a2a36faed16e2c65df7.jpg,0
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,10509,train_images\00117e4fc239b1b641ff08340b429633.jpg,1
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,train_images\00136d1cf4edede0203f32f05f660588.jpg,3


In [5]:
train_steps={}
valid_steps={}
for fold in range(5):
    train_steps[fold]=df.loc[df['gfold']!=fold].reset_index(drop=True).shape[0]//BATCH_SIZE
    valid_steps[fold]=df.loc[df['gfold']==fold].reset_index(drop=True).shape[0]//BATCH_SIZE
train_steps,valid_steps

({0: 107, 1: 107, 2: 107, 3: 107, 4: 107}, {0: 26, 1: 26, 2: 26, 3: 26, 4: 26})

In [6]:
from transformers import TFBertModel,TFRobertaModel
from transformers import BertTokenizer,RobertaTokenizer

In [7]:
if PRE_TRAINED_NAME.startswith("bert"):
    TOKENIZER= BertTokenizer.from_pretrained(PRE_TRAINED_NAME,do_lower_case=True)
else:
    TOKENIZER= RobertaTokenizer.from_pretrained(PRE_TRAINED_NAME,do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model

In [9]:
class ARCFACE_LAYER(tf.keras.layers.Layer):
    def __init__(self,m=0.5,s=30,n_classes=11014):
        super(ARCFACE_LAYER,self).__init__()
        self.m=m
        self.s=s
        self.sin_m=tf.sin(m)
        self.cos_m=tf.cos(m)
        self.n_classes=n_classes
        self.threshold = tf.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def build(self,input_shape):
        prev_layer_units=input_shape[0][1]
        self.w=self.add_weight(shape=(prev_layer_units,self.n_classes),trainable=True,
                              initializer='glorot_uniform')

    def get_config(self):
        config=super().get_config()
        config.update({"m":0.5,
                       "s":30,
                       "n_classes":11014})
        return config


    def call(self,inputs):
        prev_layer,y=inputs
        y=tf.cast(y,dtype=tf.int32)
        y_hot=tf.one_hot(y,self.n_classes)
        y_hot=tf.cast(y_hot,dtype=tf.float32)
        w_norm=tf.linalg.l2_normalize(self.w,axis=0)
        x_norm=tf.linalg.l2_normalize(prev_layer,axis=1)
        cos_theta=tf.linalg.matmul(x_norm,w_norm)
        sin_theta=tf.sqrt(1-tf.pow(cos_theta,tf.cast(2,dtype=tf.float32)))
        cos_theta_m=(cos_theta*self.cos_m)-(sin_theta*self.sin_m)
        cos_theta_m=tf.where(cos_theta>self.threshold,cos_theta_m,cos_theta-self.mm)
        final=self.s*((y_hot*cos_theta_m)+((1-y_hot)*cos_theta))
        return final

In [10]:
def MAKE_MODEL_INPUTS(dataframe):
    INPUT_IDS=np.zeros((dataframe.shape[0],MAX_LENGTH),dtype=np.int32)
    ATTENTION_MASK=np.zeros((dataframe.shape[0],MAX_LENGTH),dtype=np.int32)
    if PRE_TRAINED_NAME.startswith("bert"):
        TOKEN_TYPE_IDS=np.zeros((dataframe.shape[0],MAX_LENGTH),dtype=np.int32)
    Y=np.zeros((dataframe.shape[0],),dtype=np.int32)
    n_iter=int(np.ceil(dataframe.shape[0])/BATCH_SIZE)
    max_size=dataframe.shape[0]
    for batch in range(n_iter):
        start=batch*BATCH_SIZE
        end=(batch+1)*BATCH_SIZE
        if end>max_size:
            end=max_size
        if PRE_TRAINED_NAME.startswith("bert"):
            curr=dataframe.iloc[start:end]['title'].values.tolist()
        else:
            curr=list(" "+x for x in dataframe.iloc[start:end]['title'].values.tolist())
        tokenized_data=TOKENIZER.batch_encode_plus(curr,max_length=MAX_LENGTH,truncation="longest_first",padding='max_length')
        INPUT_IDS[start:end,]=tokenized_data['input_ids']
        ATTENTION_MASK[start:end,]=tokenized_data['attention_mask']
        if PRE_TRAINED_NAME.startswith("bert"):
            TOKEN_TYPE_IDS[start:end,]=tokenized_data['token_type_ids']
        Y[start:end,]=dataframe.iloc[start:end]['label_group'].values.tolist()
    MODEL_INPUTS=({"input_ids":INPUT_IDS,"attention_mask":ATTENTION_MASK},Y)
    if PRE_TRAINED_NAME.startswith("bert"):
        MODEL_INPUTS[0]['token_type_ids']=TOKEN_TYPE_IDS
    MODEL_OUTPUTS=Y
    return MODEL_INPUTS,MODEL_OUTPUTS

In [11]:
#for MAX_LENGTH in [120,180,250,270]:
    #print(f"now runnig for max_length : {MAX_LENGTH}")
for fold in range(5):
    train_data=df.loc[df['gfold']!=fold].reset_index(drop=True)
    test_data=df.loc[df['gfold']==fold].reset_index(drop=True)
    MODEL_INPUTS,MODEL_OUTPUTS=MAKE_MODEL_INPUTS(train_data)
    VALID_INPUTS,VALID_OUTPUTS=MAKE_MODEL_INPUTS(test_data)
    del train_data,test_data
    import gc
    gc.collect()
    tf.keras.backend.clear_session()
    with strategy.scope():
        input_ids=Input((MAX_LENGTH,),dtype=tf.int32)
        attention_mask=Input((MAX_LENGTH,),dtype=tf.int32)
        if PRE_TRAINED_NAME.startswith("bert"):
            token_type_ids=Input((MAX_LENGTH,),dtype=tf.int32)
        ins=Input((),name="label_input")
        if PRE_TRAINED_NAME.startswith("bert"):
            pre_trained=TFBertModel.from_pretrained(PRE_TRAINED_NAME,output_hidden_states=True)
            pre_outputs=pre_trained({"input_ids":input_ids
                                     ,"attention_mask":attention_mask
                                    ,"token_type_ids":token_type_ids})
        else:
            pre_trained=TFRobertaModel.from_pretrained(PRE_TRAINED_NAME,output_hidden_states=True)
            pre_outputs=pre_trained({"input_ids":input_ids
                                     ,"attention_mask":attention_mask})
        hidden_layers=[]
        k=0
        for i in reversed(range(len(pre_outputs['hidden_states']))):
            if k<4:
                hidden_layers.append(pre_outputs['hidden_states'][i])
                k+=1
            else:
                break
        x=tf.keras.layers.Concatenate()(hidden_layers)[:,0,:]
        x=Dense(720)(x)
        arc_layer=ARCFACE_LAYER()
        x=arc_layer([x,ins])
        outs=tf.keras.layers.Softmax()(x)
        if PRE_TRAINED_NAME.startswith("bert"):
            model=Model(inputs=({"input_ids":input_ids,"attention_mask":attention_mask,
                                "token_type_ids":token_type_ids},ins),outputs=outs)
        else:
            model=Model(inputs=({"input_ids":input_ids,"attention_mask":attention_mask},ins),outputs=outs)
        print(f"training for fold {fold}")
        if fold==0:
            print(model.summary())
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    NUM_TRAIN_STEPS=train_steps[fold]
    NUM_VALID_STEPS=valid_steps[fold]
    early=tf.keras.callbacks.EarlyStopping(monitor="val_loss",mode="min",verbose=1,patience=10)
    saver=tf.keras.callbacks.ModelCheckpoint(filepath=PRE_TRAINED_NAME+f"{fold}_{MAX_LENGTH}.h5",
                                     monitor="val_loss",mode="min",save_best_only=True,
                                     save_weights_only=True,verbose=1)
    model.fit(MODEL_INPUTS,MODEL_OUTPUTS,
              validation_data=(VALID_INPUTS,VALID_OUTPUTS),epochs=80,
              callbacks=[early,saver],steps_per_epoch=NUM_TRAIN_STEPS,
             validation_steps=NUM_VALID_STEPS)
    print(f"model training for {fold} is done")
    del model,MODEL_INPUTS,MODEL_OUTPUTS,VALID_INPUTS,VALID_OUTPUTS
    import gc
    gc.collect()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


training for fold 0
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 180)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 180)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 180)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_2[0][0]                    
                                                                 input_1[0

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

training for fold 2
Epoch 1/80
107/107 [==============================] - 163s 775ms/step - loss: 24.0862 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 22.6724 - val_sparse_categorical_accuracy: 0.0283

Epoch 00001: val_loss improved from inf to 22.67242, saving model to bert-base-uncased2_180.h5
Epoch 2/80
107/107 [==============================] - 22s 203ms/step - loss: 22.5482 - sparse_categorical_accuracy: 0.0010 - val_loss: 20.6891 - val_sparse_categorical_accuracy: 0.0375

Epoch 00002: val_loss improved from 22.67242 to 20.68905, saving model to bert-base-uncased2_180.h5
Epoch 3/80
107/107 [==============================] - 22s 204ms/step - loss: 20.4262 - sparse_categorical_accuracy: 0.0099 - val_loss: 19.1139 - val_sparse_categorical_accuracy: 0.0561

Epoch 00003: val_loss improved from 20.68905 to 19.11388, saving model to bert-base-uncased2_180.h5
Epoch 4/80
107/107 [==============================] - 22s 203ms/step - loss: 18.5911 - sparse_categorical_accuracy: 0.0238

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


training for fold 3
Epoch 1/80
107/107 [==============================] - 165s 777ms/step - loss: 24.0790 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 22.6930 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 22.69299, saving model to bert-base-uncased3_180.h5
Epoch 2/80
107/107 [==============================] - 22s 204ms/step - loss: 22.4808 - sparse_categorical_accuracy: 0.0010 - val_loss: 20.6945 - val_sparse_categorical_accuracy: 0.0393

Epoch 00002: val_loss improved from 22.69299 to 20.69453, saving model to bert-base-uncased3_180.h5
Epoch 3/80
107/107 [==============================] - 22s 204ms/step - loss: 20.4076 - sparse_categorical_accuracy: 0.0103 - val_loss: 19.2076 - val_sparse_categorical_accuracy: 0.0543

Epoch 00003: val_loss improved from 20.69453 to 19.20756, saving model to bert-base-uncased3_180.h5
Epoch 4/80
107/107 [==============================] - 22s 203ms/step - loss: 18.5274 - sparse_categorical_accuracy: 0.

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


training for fold 4
Epoch 1/80
107/107 [==============================] - 168s 773ms/step - loss: 24.0845 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 22.8582 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 22.85823, saving model to bert-base-uncased4_180.h5
Epoch 2/80
107/107 [==============================] - 22s 204ms/step - loss: 22.6516 - sparse_categorical_accuracy: 6.5273e-04 - val_loss: 20.9573 - val_sparse_categorical_accuracy: 0.0358

Epoch 00002: val_loss improved from 22.85823 to 20.95729, saving model to bert-base-uncased4_180.h5
Epoch 3/80
107/107 [==============================] - 22s 204ms/step - loss: 20.6112 - sparse_categorical_accuracy: 0.0095 - val_loss: 19.4571 - val_sparse_categorical_accuracy: 0.0492

Epoch 00003: val_loss improved from 20.95729 to 19.45710, saving model to bert-base-uncased4_180.h5
Epoch 4/80
107/107 [==============================] - 22s 203ms/step - loss: 18.7604 - sparse_categorical_accuracy

In [ ]:
9.07633,8.76228,8.63512,8.94175,8.93844

In [ ]:
for 40 epochs
#Bert
base
3e-5
hidden states
512->10.57901
720->10.73882
pooler->
512->11.51173
no dense->20.94596
##########################
5e-5
hidden states
512->10.01861
720->9.84583
#########
Roberta
3e-4 15.29412
3e-5
512->12.12880
720->
1e-5
512->16.21013
720->
5e-5
512->11.14457
720->11.04617
########
XLnet(hidden states not good)
2e-5
with token_type->13.98930
without token_type->14.00980
##############################################################################
bert and roberta with 720,5e-5
Roberta
max_length->120,180,250,270 (error rates) for 80 epochs
10.83226,10.72349,10.59485,10.67350
Bert
max_length->120,180,250,270 (error rates) for 80 epochs
8.99459,8.85223,8.92365,9.06225
###################


Best chosen is Bert,180,720,5e-5